## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,Qaanaaq,77.4840,-69.3632,9.14,76,87,3.36,GL,2021-01-15 02:15:11
1,1,Ushuaia,-54.8000,-68.3000,46.40,71,75,12.66,AR,2021-01-15 02:17:24
2,2,Luwuk,-0.9516,122.7875,85.71,70,68,1.70,ID,2021-01-15 02:25:44
3,3,Whitehorse,60.7161,-135.0538,23.00,93,90,12.66,CA,2021-01-15 02:17:48
4,4,Clevelandia,-26.3958,-52.4708,64.92,82,0,7.87,BR,2021-01-15 02:25:44


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,2,Luwuk,-0.9516,122.7875,85.71,70,68,1.70,ID,2021-01-15 02:25:44
5,5,Sorong,-0.8833,131.2500,83.61,74,100,5.41,ID,2021-01-15 02:25:44
7,7,Hilo,19.7297,-155.0900,82.40,57,1,9.22,US,2021-01-15 02:17:18
8,8,Flinders,-34.5833,150.8552,81.00,66,17,4.00,AU,2021-01-15 02:18:27
14,14,Kapaa,22.0752,-159.3190,79.00,73,1,8.05,US,2021-01-15 02:17:24
16,16,Lethem,3.3803,-59.7968,79.34,68,92,11.79,GY,2021-01-15 02:25:45
18,18,Atuona,-9.8000,-139.0333,77.68,74,0,11.36,PF,2021-01-15 02:15:11
19,19,Rikitea,-23.1203,-134.9692,78.42,72,13,10.11,PF,2021-01-15 02:17:15
20,20,Maceio,-9.6658,-35.7353,77.00,88,40,3.44,BR,2021-01-15 02:17:48
27,27,Grenville,12.1167,-61.6167,78.80,83,20,17.27,GD,2021-01-15 02:25:46


In [7]:
# 4a. Determine if there are any empty rows.
print(len(preferred_cities_df))
preferred_cities_df.count()

171


City_ID       171
City          171
Lat           171
Lng           171
Max Temp      171
Humidity      171
Cloudiness    171
Wind Speed    171
Country       171
Date          171
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna(axis="index", how="any")

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Lat,Lng,Hotel Name
2,Luwuk,ID,85.71,-0.9516,122.7875,
5,Sorong,ID,83.61,-0.8833,131.2500,
7,Hilo,US,82.40,19.7297,-155.0900,
8,Flinders,AU,81.00,-34.5833,150.8552,
14,Kapaa,US,79.00,22.0752,-159.3190,
16,Lethem,GY,79.34,3.3803,-59.7968,
18,Atuona,PF,77.68,-9.8000,-139.0333,
19,Rikitea,PF,78.42,-23.1203,-134.9692,
20,Maceio,BR,77.00,-9.6658,-35.7353,
27,Grenville,GD,78.80,12.1167,-61.6167,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"

    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json" 

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
   
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...skipping the city.")

Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.


In [11]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(axis="index", how="any")
clean_hotel_df.head(15)

,City,Country,Max Temp,Lat,Lng,Hotel Name
2,Luwuk,ID,85.71,-0.9516,122.7875,Hotel Santika Luwuk
5,Sorong,ID,83.61,-0.8833,131.2500,The Waigo Hotel
7,Hilo,US,82.40,19.7297,-155.0900,Hilo Hawaiian Hotel
8,Flinders,AU,81.00,-34.5833,150.8552,Shellharbour Resort & Conference Centre
14,Kapaa,US,79.00,22.0752,-159.3190,Sheraton Kauai Resort at Coconut Beach
16,Lethem,GY,79.34,3.3803,-59.7968,Savannah Inn
18,Atuona,PF,77.68,-9.8000,-139.0333,Villa Enata
19,Rikitea,PF,78.42,-23.1203,-134.9692,Pension Maro'i
20,Maceio,BR,77.00,-9.6658,-35.7353,Hotel Ponta Verde Maceió
27,Grenville,GD,78.80,12.1167,-61.6167,Valley Breeze Guest House


In [12]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))